In [1]:
%pylab
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from metric import tests
import cv2
from utils import wand_transforms
import wand
from wand.image import Image
from metric.separate_iou import siou
from sklearn.metrics import adjusted_rand_score

In [3]:
#generate random images 100
!mkdir expr2_data

In [3]:
for i in range(100):
    N = randint(19, 23, dtype=int)
    circs = tests.uniform_random_circles(25, 55, 450, 10, 50, 5)
    slices = np.zeros((N, 512, 512), np.uint8)
    image = np.zeros((512, 512), np.uint8)
    for j, (c, r) in enumerate(circs):
        if j < N:
            slices[j] = cv2.circle(slices[j], c, r, 1, -1)
        image = cv2.circle(image, c, r, 255, -1)
    cv2.imwrite(f'./expr2_data/image_{i}.png', image)
    np.save(f'./expr2_data/slices_{i}.npy', slices)

## Sanity check

In [3]:
ious = []
nious = []
aris = []
fg_aris = []
fgfg_aris = []
for i in range(100):
    slc = np.load(f'./expr2_data/slices_{i}.npy')
    img = cv2.imread(f'./expr2_data/image_{i}.png', cv2.IMREAD_ANYDEPTH)
    img = (img > 127).astype(uint8)
    ious.append(siou(slc, img))
    
    orig = slc.sum(0).astype(uint8)
#     tocompare = np.ones((512, 1026), uint8)
#     tocompare[:, :512] = orig
#     tocompare[:, 514:] = img
#     cv2.imwrite(f'./expr2_data/compare/{i}.png', tocompare*255)
    
    _, x = cv2.connectedComponents(orig, connectivity=8)
    _, y = cv2.connectedComponents(img, connectivity=8)
    aris.append(adjusted_rand_score(x.reshape(-1), y.reshape(-1)))
    
    idx = (orig != 0)
    fg_aris.append(adjusted_rand_score(x[idx].reshape(-1), y[idx].reshape(-1)))
    
    idx = (orig != 0) | (y != 0)
    fgfg_aris.append(adjusted_rand_score(x[idx].reshape(-1), y[idx].reshape(-1)))
    
    orig = orig.astype(bool)
    img = img.astype(bool)
    nious.append((orig&img).sum()/(orig|img).sum())
ious = array(ious)
nious = array(nious)
aris = array(aris)
fg_aris = array(fg_aris)
fgfg_aris = array(fgfg_aris)
print(ious.mean(), ious.std())
print(nious.mean(), nious.std())
print(aris.mean(), aris.std())
print(fg_aris.mean(), fg_aris.std())
print(fgfg_aris.mean(), fgfg_aris.std())

1.0 0.0
0.8779530104190476 0.056906166620670384
0.907890210984392 0.04428963385345463
1.0 0.0
0.9139304696526054 0.07485331624956607


## Perturbed versions

In [4]:
tsfm = wand_transforms.Distort(4, 512, 10)
sious = []
nious = []
aris = []
fg_aris = []
fgfg_aris = []

for i in range(100):
    img = Image(filename=f'./expr2_data/image_{i}.png')
    slc = np.load(f'./expr2_data/slices_{i}.npy')
    img, *_ = tsfm([img])
    img = np.frombuffer(img.make_blob('gray'), uint8).reshape(512,512)
    img = (img > 127).astype(uint8)
    sious.append(siou(slc, img))
    
    orig = slc.sum(0).astype(uint8)
    _, x = cv2.connectedComponents(orig, connectivity=8)
    _, y = cv2.connectedComponents(img, connectivity=8)
    aris.append(adjusted_rand_score(x.reshape(-1), y.reshape(-1)))

    idx = (orig != 0)
    fg_aris.append(adjusted_rand_score(x[idx].reshape(-1), y[idx].reshape(-1)))
    
    idx = (orig != 0) | (y != 0)
    fgfg_aris.append(adjusted_rand_score(x[idx].reshape(-1), y[idx].reshape(-1)))

    orig = orig.astype(bool)
    img = img.astype(bool)
    nious.append((orig&img).sum()/(orig|img).sum())
sious = array(sious)
nious = array(nious)
aris = array(aris)
fg_aris = array(fg_aris)
fgfg_aris = array(fgfg_aris)
print(sious.mean(), sious.std())
print(nious.mean(), nious.std())
print(aris.mean(), aris.std())
print(fg_aris.mean(), fg_aris.std())
print(fgfg_aris.mean(), fgfg_aris.std())

0.9429138711279608 0.0350454031615335
0.7367413917992632 0.05357384485790448
0.787102979315215 0.04460379534960318
0.8697330039263061 0.03542776413437103
0.6385445882830613 0.08608864159362875
